In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
!pip install contractions
!pip install emoji
nltk.download('stopwords')
nltk.download('punkt_tab')
nltk.download('punkt')
nltk.download('wordnet')

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 345.1/345.1 kB 5.9 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 113.9/113.9 kB 4.9 MB/s eta 0:00:00


[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to /usr/share/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [3]:
#importing all the necessary libraries
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import nltk
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
import re
import string
from tqdm import tqdm
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize,sent_tokenize
from nltk.stem import PorterStemmer,WordNetLemmatizer
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
import emoji
import contractions

In [4]:
df=pd.read_csv("/kaggle/input/sentiment1/train.csv",header=None)
columns=["Label","Review_heading","Review_body"]
df.columns=columns

In [5]:
df.shape

(3600000, 3)

In [6]:
df.head()

,Label,Review_heading,Review_body
0,2,Stuning even for the non-gamer,This sound track was beautiful! It paints the ...
1,2,The best soundtrack ever to anything.,I'm reading a lot of reviews saying that this ...
2,2,Amazing!,This soundtrack is my favorite music of all ti...
3,2,Excellent Soundtrack,I truly like this soundtrack and I enjoy video...
4,2,"Remember, Pull Your Jaw Off The Floor After He...","If you've played the game, you know how divine..."


In [7]:
df.isnull().sum()

Label               0
Review_heading    207
Review_body         0
dtype: int64

In [8]:
df.drop(['Review_heading'],axis=1,inplace=True)

In [9]:
df.head()

,Label,Review_body
0,2,This sound track was beautiful! It paints the ...
1,2,I'm reading a lot of reviews saying that this ...
2,2,This soundtrack is my favorite music of all ti...
3,2,I truly like this soundtrack and I enjoy video...
4,2,"If you've played the game, you know how divine..."


In [10]:
stop_words=set(stopwords.words('english'))
lemma=WordNetLemmatizer()
negation_words = {"no", "nor", "not", "ain", "aren't", "couldn't", "didn't", "doesn't","hadn't", "hasn't", "haven't", "isn't", "shouldn't", "wasn't","weren't", "won't", "wouldn't","mightn't","needn't"}
stop_words = stop_words-negation_words
correction_dict = {'bday': 'birthday', 'gr8': 'great', 'luv': 'love', 'ur': 'your', 'pls': 'please', 'thx': 'thanks', 'u': 'you', 'brb': 'be right back', 'idk': 'I don\'t know', 'omg': 'oh my god', 'lol': 'laugh out loud', 'tbh': 'to be honest', 'fyi': 'for your information', 'lmk': 'let me know', 'btw': 'by the way', 'asap': 'as soon as possible', 'smh': 'shaking my head', 'ttyl': 'talk to you later', 'ppl': 'people', 'nvm': 'never mind', 'cya': 'see you', 'rofl': 'rolling on the floor laughing', 'omw': 'on my way', 'wdym': 'what do you mean', 'fomo': 'fear of missing out', 'yolo': 'you only live once', 'lmao': 'laughing my ass off', 'gtg': 'got to go', 'wbu': 'what about you', 'bbl': 'be back later', 'bff': 'best friends forever', 'gm': 'good morning', 'gn': 'good night', 'np': 'no problem', 'gg': 'good game', 'afk': 'away from keyboard', 'yup': 'yes', 'nah': 'no', 'yass': 'yes', 'plz': 'please', 'thru': 'through', 'gr8t': 'great', 'wat': 'what', 'wht': 'what', 'howdy': 'hello', 'g2g': 'got to go', 'l8r': 'later', 'no1': 'no one', 'cuz': 'because', 'bro': 'brother', 'sis': 'sister', 'imho': 'in my humble opinion', 'ftw': 'for the win', 'tmi': 'too much information', 'jmho': 'just my humble opinion', 'tbh': 'to be honest', 'btw': 'by the way', 'jk': 'just kidding', 'afaik': 'as far as I know', 'ik': 'I know', 'wfh': 'work from home', 'lmk': 'let me know', 'swag': 'style, confidence', 'fam': 'family', 'thnx': 'thanks', 'gr8ful': 'grateful', 'wyd': 'what you doing', 'sd': 'social distancing', 'pplz': 'people', 'seeya': 'see you', 'yay': 'yes', 'hbu': 'how about you', 'tho': 'though', 'm8': 'mate', 'gr8ful': 'grateful', 'gimme': 'give me', 'fml': 'f**k my life', 'qik': 'quick', 'realy': 'really', 'yr': 'your', 'wtf': 'what the f**k', 'bffl': 'best friends for life', '2morrow': 'tomorrow', '2nite': 'tonight', 'wth': 'what the hell', 'stfu': 'shut the f**k up', 'ngl': 'not gonna lie', 'tbh': 'to be honest', 'smh': 'shaking my head', 'hbd': 'happy birthday', 'gg': 'good game', 'n00b': 'newbie', 'pmu': 'pissed me off', 'rotfl': 'rolling on the floor laughing', 'sol': 'shout out loud', 'omfg': 'oh my f**king god', 'srsly': 'seriously', 'dunno': 'don\'t know', 'bbl': 'be back later', 'lolz': 'laugh out loud', 'l8': 'late', 'fr': 'for real', 'plz': 'please', 'stoked': 'excited', 'lit': 'awesome', 'noob': 'newbie', 'h8': 'hate', 'xoxo': 'hugs and kisses', 'smh': 'shaking my head', 'yolo': 'you only live once','plz':'please','gn':'good night'}


#nlp=spacy.load("en_core_web_lg",disable=['ner','parse'])
tqdm.pandas()
def remove_HTML_tags(text):
    pattern = re.compile('<.*?>')
    return re.sub(pattern, '', text)
def lowercasing(text):
    return text.lower()
def remove_URL(text):
    pattern = re.compile(r'https?://\S+|www\.\S+')
    return re.sub(pattern, '', text)
def remove_punc(text):
    exclude = string.punctuation
    return text.translate(str.maketrans('', '', exclude))
def spell_correction(text):
    textblb = TextBlob(text)
    return textblb.correct().string
def remove_emoji(text):
    emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"  # Emoticons
                               u"\U0001F300-\U0001F5FF"  # Miscellaneous Symbols and Pictographs
                               u"\U0001F680-\U0001F6FF"  # Transport and Map Symbols
                               u"\U0001F1E0-\U0001F1FF"  # Regional Indicator Symbols
                               u"\U00002702-\U000027B0"  # Dingbats
                               u"\U000024C2-\U0001F251"  # Enclosed characters
                               "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', text)
def demojify(text):
    return emoji.demojize(text)
def expand_contractions(text):
    return contractions.fix(text)
def remove_stopwords(text):
    list_words = [word for word in word_tokenize(text) if word not in stop_words]
    return ' '.join(list_words)
def remove_extra_whitespaces(text):
    return re.sub(r'\s+', ' ', text).strip()
def lemmatization(text):
    l=[lemma.lemmatize(word) for word in word_tokenize(text)]
    return ' '.join(l)
def replace_slang_in_review(text):
    words = word_tokenize(text)
    corrected_text = [correction_dict[word] if word in correction_dict else word for word in words]
    return ' '.join(corrected_text)
tqdm.pandas()

df['Review_body'] = df['Review_body'].progress_apply(lowercasing)
df['Review_body'] = df['Review_body'].progress_apply(remove_extra_whitespaces)
df['Review_body'] = df['Review_body'].progress_apply(remove_HTML_tags)
df['Review_body'] = df['Review_body'].progress_apply(remove_URL)
df['Review_body'] = df['Review_body'].progress_apply(remove_punc)
df['Review_body'] = df['Review_body'].progress_apply(expand_contractions)
df['Review_body'] = df['Review_body'].progress_apply(replace_slang_in_review)
df['Review_body'] = df['Review_body'].progress_apply(demojify)  # OR remove_emoji
df['Review_body'] = df['Review_body'].progress_apply(remove_stopwords)
df['Review_body'] = df['Review_body'].progress_apply(lemmatization)

100%|██████████| 3600000/3600000 [21:30<00:00, 2789.11it/s]


In [14]:
df.duplicated().sum()

5884

In [15]:
df = df.drop_duplicates()

In [17]:
df=df.reset_index()

In [20]:
df=df.drop(['index'],axis=1)

In [21]:
df

,Label,Review_body
0,2,sound track beautiful paint senery mind well w...
1,2,reading lot review saying best game soundtrack...
2,2,soundtrack favorite music time hand intense sa...
3,2,truly like soundtrack enjoy video game music p...
4,2,played game know divine music every single son...
...,...,...
3594111,1,high chair look great first come box hill impo...
3594112,1,used highchair 2 kid finally decided sell not ...
3594113,1,small house really wanted two high chair twin ...
3594114,1,not sure book supposed really rehash old idea ...


In [22]:
df=df[:3500000]

In [23]:
df

,Label,Review_body
0,2,sound track beautiful paint senery mind well w...
1,2,reading lot review saying best game soundtrack...
2,2,soundtrack favorite music time hand intense sa...
3,2,truly like soundtrack enjoy video game music p...
4,2,played game know divine music every single son...
...,...,...
3499995,1,ordered prouct 510 course charge card hit cost...
3499996,1,bought 45 rating disappointed performancethe p...
3499997,2,xpad everything expected nonslip surface elimi...
3499998,2,using xpad 3 month must say xpad best purchase...


In [24]:
df["Label"].value_counts()

Label
2    1751563
1    1748437
Name: count, dtype: int64

In [25]:
df.to_csv('/kaggle/working/sentipreprocessed.csv', index=False)